In [ ]:
%%capture
!pip install asteroid

In [ ]:
%%capture
!pip install pytorch-lightning scikit-plot neptune-client neptune-contrib[viz] --upgrade

In [ ]:
%%capture
!pip install neptune-client

# [Conv-TasNet: Surpassing Ideal Time-Frequency Magnitude Masking for Speech Separation](https://arxiv.org/pdf/1809.07454.pdf)

Documentation: https://asteroid.readthedocs.io/en/latest/why_use_asteroid.html

1.   https://github.com/asteroid-team/asteroid/blob/master/egs/librimix/ConvTasNet/train.py

2. CSV: https://drive.google.com/drive/u/0/folders/1-9KimCUNx_tnVk87iOXMKzlbeYuJ4Ba5

3. DataLoader : https://github.com/asteroid-team/asteroid/blob/e7cb31ec2fdb24aa185e4fd1c5c73a130fb66c33/asteroid/data/librimix_dataset.py#L17

4. System o modelo pytorch Lightning: https://github.com/asteroid-team/asteroid/blob/master/asteroid/engine/system.py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/TG-Separación-Fuentes/code/train-speech-separation-models/train

/content/drive/Shareddrives/TG-Separación-Fuentes/code/train-speech-separation-models/train


In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
"""
%%capture
!git clone https://github.com/pytorch/audio.git
os.chdir("audio")
!git checkout 301e2e9
!python setup.py install
"""

'\n%%capture\n!git clone https://github.com/pytorch/audio.git\nos.chdir("audio")\n!git checkout 301e2e9\n!python setup.py install\n'

In [ ]:
!nvidia-smi

Mon Jun 28 16:55:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from torch.optim import Adam
from torch.utils.data import DataLoader,Dataset
import pytorch_lightning as pl
import yaml
import json
from asteroid import ConvTasNet
from asteroid.utils import prepare_parser_from_dict, parse_args_as_dict
import argparse
from torch.optim.lr_scheduler import ReduceLROnPlateau
from asteroid.metrics import get_metrics
from asteroid.utils import tensors_to_device
from tqdm import tqdm
from asteroid.dsp.normalization import normalize_estimates

import os
from asteroid.engine.system import System
from asteroid.losses import PITLossWrapper, pairwise_neg_sisdr
import soundfile as sf
import torch
import random as random
from IPython.display import display, Audio
from asteroid.models import BaseModel
from pytorch_lightning.callbacks import ModelCheckpoint,EarlyStopping
from asteroid.engine.optimizers import make_optimizer
from pytorch_lightning.loggers.neptune import NeptuneLogger

In [ ]:
%cd /content/drive/Shareddrives/TG-Separación-Fuentes/code/train-speech-separation-models/train
from lib.CallSpanish_dataset import CallSpanish

/content/drive/Shareddrives/TG-Separación-Fuentes/code/train-speech-separation-models/train


# 1.1 Configuración de variables globales

In [ ]:
# Save checkpoints
default_root_dir = "/content/drive/Shareddrives/TG-Separación-Fuentes/code/Checkpoints-separation-models/ConvTasnet/best_model_train_all_dataset_200_epochs_Transfer_L"
save_best_model = "/content/drive/Shareddrives/TG-Separación-Fuentes/code/Checkpoints-separation-models/ConvTasnet/best_model_train_all_dataset_200_epochs_Transfer_L/"

In [ ]:
PATH_DATA_ROOT = "../../Datasets/CallFriend-Spanish-Prepro/Train/"
MIX = PATH_DATA_ROOT+"mix/"
S_1 = PATH_DATA_ROOT+"source_1/"
S_2 = PATH_DATA_ROOT+"source_2/"



# CSV
ROOT_CSV  ="/content/drive/Shareddrives/TG-Separación-Fuentes/Datasets/Datasets-CSV/"
#ROOT_CSV = "../resources/"

PATH_CSV_MIX = ROOT_CSV +"mixture_callfriend_spanish.csv"
PATH_CSV_TRAIN = ROOT_CSV + "mixture_train_mix_clean_callfriend_spanish_15_seconds.csv"
PATH_CSV_VALID = ROOT_CSV +"mixture_val_mix_clean_callfriend_spanish_15_seconds.csv"
PATH_CSV_TEST = ROOT_CSV + "mixture_test_mix_clean_callfriend_spanish_15_seconds.csv"


PATH_CONFIG = "./resources/conf.yml"

df_train = pd.read_csv(PATH_CSV_TRAIN)
df_val = pd.read_csv(PATH_CSV_VALID)
df_test = pd.read_csv(PATH_CSV_TEST)

In [ ]:
ls -l ../../../Datasets/Datasets-CSV/

total 15983
-rw------- 1 root root  838698 Jun 26 04:07 ALL_mixture_test_mix_clean_spanish.csv
-rw------- 1 root root 2957042 Jun 26 04:06 ALL_mixture_train_mix_clean_spanish.csv
-rw------- 1 root root  337520 Jun 26 04:09 ALL_mixture_val_mix_clean_spanish.csv
drwx------ 2 root root    4096 Jun 20 17:05 callfriend_spanish_5_seconds/
-rw------- 1 root root  656353 Jun 21 04:16 mixture_test_mix_clean_callfriend_spanish_15_seconds.csv
-rw------- 1 root root  262778 Jun 24 17:49 mixture_test_mix_clean_callfriend_spanish_caribbean.csv
-rw------- 1 root root  319427 Jun 20 21:55 mixture_test_mix_clean_callfriend_spanish.csv
-rw------- 1 root root  256611 Jun 25 15:31 mixture_test_mix_clean_callHome_spanish.csv
-rw------- 1 root root 2211217 Jun 21 04:16 mixture_train_mix_clean_callfriend_spanish_15_seconds.csv
-rw------- 1 root root  827678 Jun 24 17:49 mixture_train_mix_clean_callfriend_spanish_caribbean.csv
-rw------- 1 root root 1076827 Jun 20 21:55 mixture_train_mix_clean_callfriend_span

In [ ]:
df_train.shape,df_val.shape,df_test.shape

((6174, 5), (804, 5), (1848, 5))

In [ ]:
sum(df_test.length.map(lambda x:x/8000))/3600

7.684381944444445

In [ ]:
with open(PATH_CONFIG) as f:
        conf = yaml.safe_load(f)
conf["main_args"]={"exp_dir":save_best_model}
exp_dir = save_best_model

In [ ]:
conf

{'data': {'n_src': 2,
  'sample_rate': 8000,
  'segment': 3,
  'task': 'sep_clean',
  'train_dir': 'data/wav8k/min/train-100',
  'valid_dir': 'data/wav8k/min/dev'},
 'filterbank': {'kernel_size': 16, 'n_filters': 512, 'stride': 8},
 'main_args': {'exp_dir': '/content/drive/Shareddrives/TG-Separación-Fuentes/code/Checkpoints-separation-models/ConvTasnet/best_model_train_all_dataset_200_epochs_Transfer_L/'},
 'masknet': {'bn_chan': 128,
  'hid_chan': 512,
  'mask_act': 'relu',
  'n_blocks': 8,
  'n_repeats': 3,
  'skip_chan': 128},
 'optim': {'lr': 0.001, 'optimizer': 'adam', 'weight_decay': 0.0},
 'test': {'n_save_examples': 5},
 'training': {'batch_size': 6,
  'early_stop': True,
  'epochs': 200,
  'half_lr': True,
  'num_workers': 4}}

In [ ]:
df_train.shape

(8180, 5)

In [ ]:
start = 0
end = df_test.shape[0]

In [ ]:
#conf["training"]["epochs"] = 200

Epoch = conf["training"]["epochs"]

In [ ]:
Epoch

200

In [ ]:
best_epoch = 63

In [ ]:
# Configurarmos el experimento y sus parametros
experiment_name = "Test_30_seconds_ConvTasnet_26.4_train_2.9_val_7.3_test_sta_"+str(start)+"_end_"+str(end)
params=conf
tags = ["30_seconds_TEST_Best_Model_lote_start_"+str(start)+"_end_"+str(end)+"_ConvTasnet__train_from_PRETRAINED_best_epoch_"+str(best_epoch)]

# Definir Logger 
neptune_logger = NeptuneLogger(
    api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI5NjRkMmY2YS04M2EwLTRiMGMtODk1Ny1mMWQxZTA3NGM1NzAifQ==",
    project_name="josearangos/Tg-speech-separation",experiment_name=experiment_name,
    params = params, tags = tags, close_after_fit=False)

NeptuneLogger will work in online mode


In [ ]:
conf["training"]["epochs"]

200

# Test modelo antes de train

Modelo pretrained

In [ ]:
model_before = BaseModel.from_pretrained("JorisCos/ConvTasNet_Libri2Mix_sepclean_8k")
model_before.cuda()

In [ ]:
PATH_TEST = df_val.iloc[310,1]  
mixture, _ = sf.read(PATH_TEST, dtype="float32", always_2d=True)
# Soundfile returns the mixture as shape (time, channels), and Asteroid expects (batch, channels, time)
mixture = mixture.transpose()
mixture = mixture.reshape(1, mixture.shape[0], mixture.shape[1])
out_wavs_before = model_before.separate(mixture)

In [ ]:
display(Audio(PATH_TEST))
display(Audio(out_wavs_before[0,0,:],rate=8000))
display(Audio(out_wavs_before[0,1,:],rate=8000))

# Calcular metricas modelo preentrenado

In [ ]:
test_set = CallSpanish(
    csv_path=PATH_CSV_TEST,
    task="sep_clean",
    sample_rate=8000,
    n_src=2,
    segment=None,
    return_id=True
)

In [ ]:
len(test_set)

2338

In [ ]:
exp_dir = conf["main_args"]["exp_dir"]
os.makedirs(exp_dir, exist_ok=True)

In [ ]:
path_best_model = os.path.join(exp_dir, "best_model.pth")
best_model  = ConvTasNet.from_pretrained(path_best_model)
best_model.cuda()

ConvTasNet(
  (encoder): Encoder(
    (filterbank): FreeFB()
  )
  (masker): TDConvNet(
    (bottleneck): Sequential(
      (0): GlobLN()
      (1): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
    )
    (TCN): ModuleList(
      (0): Conv1DBlock(
        (shared_block): Sequential(
          (0): Conv1d(128, 512, kernel_size=(1,), stride=(1,))
          (1): PReLU(num_parameters=1)
          (2): GlobLN()
          (3): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,), groups=512)
          (4): PReLU(num_parameters=1)
          (5): GlobLN()
        )
        (res_conv): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
        (skip_conv): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
      )
      (1): Conv1DBlock(
        (shared_block): Sequential(
          (0): Conv1d(128, 512, kernel_size=(1,), stride=(1,))
          (1): PReLU(num_parameters=1)
          (2): GlobLN()
          (3): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), gro

In [ ]:
#Guardar las metricas 
results = np.zeros(10)
test(start, end, results, best_model,test_set,pretrained=True)

  0%|          | 0/2338 [00:00<?, ?it/s]

https://app.neptune.ai/josearangos/Tg-speech-separation/e/TGSPEEC-97


 53%|█████▎    | 1230/2338 [1:33:04<1:16:30,  4.14s/it]/usr/local/lib/python3.7/dist-packages/pystoi/stoi.py:69: RuntimeWarning: Not enough STFT frames to compute intermediate intelligibility measure after removing silent frames. Returning 1e-5. Please check you wav files
  RuntimeWarning)
100%|██████████| 2338/2338 [2:55:56<00:00,  4.52s/it]


# 1. Cargar Datos para train y valid

In [ ]:
train_set = CallSpanish(
    csv_path=PATH_CSV_TRAIN,
    task="sep_clean",
    sample_rate=8000,
    n_src=2,
    segment=3
)

val_set = CallSpanish(
   csv_path=PATH_CSV_VALID,
    task="sep_clean",
    sample_rate=8000,
    n_src=2,
    segment=3
)

Drop 5 utterances from 8180 (shorter than 3 seconds)
Drop 0 utterances from 936 (shorter than 3 seconds)


In [ ]:
len(train_set)

8175

In [ ]:
train_loader = DataLoader(train_set,shuffle=True,batch_size=4, drop_last=True,num_workers=4)
val_loader = DataLoader(val_set, batch_size=4, drop_last=True,num_workers=4)

# 2. Definición del modelo

In [ ]:
conf

{'data': {'n_src': 2,
  'sample_rate': 8000,
  'segment': 3,
  'task': 'sep_clean',
  'train_dir': 'data/wav8k/min/train-100',
  'valid_dir': 'data/wav8k/min/dev'},
 'filterbank': {'kernel_size': 16, 'n_filters': 512, 'stride': 8},
 'main_args': {'exp_dir': '/content/drive/Shareddrives/TG-Separación-Fuentes/code/Checkpoints-separation-models/ConvTasnet/best_model_train_all_dataset_200_epochs_Transfer_L/'},
 'masknet': {'bn_chan': 128,
  'hid_chan': 512,
  'mask_act': 'relu',
  'n_blocks': 8,
  'n_repeats': 3,
  'skip_chan': 128},
 'optim': {'lr': 0.001, 'optimizer': 'adam', 'weight_decay': 0.0},
 'test': {'n_save_examples': 5},
 'training': {'batch_size': 6,
  'early_stop': True,
  'epochs': 200,
  'half_lr': True,
  'num_workers': 4}}

In [ ]:
model = model_before

In [ ]:
optimizer = make_optimizer(model.parameters(), **conf["optim"])

In [ ]:
scheduler = None
if conf["training"]["half_lr"]:
        scheduler = ReduceLROnPlateau(optimizer=optimizer, factor=0.5, patience=5)
gpus = -1 if torch.cuda.is_available() else None
distributed_backend = "ddp" if torch.cuda.is_available() else None

In [ ]:
gpus

-1

In [ ]:
# Define Loss function.
loss_func = PITLossWrapper(pairwise_neg_sisdr, pit_from="pw_mtx")

In [ ]:
system = System(
        model=model,
        loss_func=loss_func,
        optimizer=optimizer,
        train_loader=train_loader,
        val_loader=val_loader,
        scheduler=scheduler,
        config=conf,
    )

In [ ]:
# Define callbacks
callbacks = []
checkpoint_dir = os.path.join(exp_dir, "checkpoints/")
checkpoint = ModelCheckpoint(checkpoint_dir, monitor="val_loss", mode="min", save_top_k=5, verbose=True)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:360: UserWarning: Checkpoint directory /content/drive/Shareddrives/TG-Separación-Fuentes/code/Checkpoints-separation-models/ConvTasnet/best_model_train_all_dataset_200_epochs_Transfer_L/checkpoints/ exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


In [ ]:
checkpoint

In [ ]:
# Stop Early
callbacks.append(checkpoint)
if conf["training"]["early_stop"]:
        callbacks.append(EarlyStopping(monitor="val_loss", mode="min", patience=30, verbose=True))

# 3. Train modelo

In [ ]:
trainer= pl.Trainer(
        max_epochs=conf["training"]["epochs"],
        callbacks=callbacks,
        default_root_dir=exp_dir,
        gpus=gpus,
        #distributed_backend=distributed_backend,
        limit_train_batches=1.0,  # Useful for fast experiment
        gradient_clip_val=5.0,
        logger=neptune_logger
        
    )

In [ ]:
trainer.fit(system)

# Guardar mejor modelo

In [ ]:
best_k = {k: v.item() for k, v in checkpoint.best_k_models.items()}
with open(os.path.join(exp_dir, "best_k_models.json"), "w") as f:
    json.dump(best_k, f, indent=0)

state_dict = torch.load(checkpoint.best_model_path)
system.load_state_dict(state_dict=state_dict["state_dict"])
system.cpu()

to_save = system.model.serialize()
best_model_path = os.path.join(exp_dir, "best_model.pth")
torch.save(to_save,best_model_path )

#Send best model to neptune
neptune_logger.experiment.log_artifact(best_model_path)

# Restaurar training

In [ ]:
model_last_epoch = save_best_model+"checkpoints/epoch=39-step=81719.ckpt"
model_last_epoch

'/content/drive/Shareddrives/TG-Separación-Fuentes/code/Checkpoints-separation-models/ConvTasnet/best_model_train_all_dataset_200_epochs_Transfer_L/checkpoints/epoch=39-step=81719.ckpt'

In [ ]:
system = System(  
        model =model,      
        loss_func=loss_func,
        optimizer=optimizer,
        train_loader=train_loader,
        val_loader=val_loader,
        scheduler=scheduler,
        config=conf,
    )

In [ ]:
trainer= pl.Trainer(
        max_epochs=conf["training"]["epochs"],
        callbacks=callbacks,
        default_root_dir=exp_dir,
        gpus=gpus,
        #distributed_backend=distributed_backend,
        limit_train_batches=1.0,  # Useful for fast experiment
        gradient_clip_val=5.0,
        logger=neptune_logger,
        resume_from_checkpoint=model_last_epoch#Restore
        
    )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(system)

# 5. Test model despues entrenar

In [ ]:
df_val.iloc[310,1]

'/content/drive/Shareddrives/TG-Separación-Fuentes/Datasets/CallFriend-Spanish-Split/valid/mix/6828_28.wav'

In [ ]:
display(Audio(PATH_TEST))
display(Audio(out_wavs_after[0,0,:],rate=8000))
display(Audio(out_wavs_after[0,1,:],rate=8000))

In [ ]:
PATH_TEST = df_val.iloc[310,1]
path_best_model =  "/content/drive/Shareddrives/TG-Separación-Fuentes/code/Checkpoints-separation-models/ConvTasnet/best_model_TGSPEEC-81/best_model.pth"#os.path.join(exp_dir, "best_model.pth")
best_model  = ConvTasNet.from_pretrained(path_best_model)
best_model.cuda()
mixture, _ = sf.read(PATH_TEST, dtype="float32", always_2d=True)
# Soundfile returns the mixture as shape (time, channels), and Asteroid expects (batch, channels, time)
mixture = mixture.transpose()
mixture = mixture.reshape(1, mixture.shape[0], mixture.shape[1])
out_wavs_after = best_model.separate(mixture)

In [ ]:
display(Audio(PATH_TEST))
display(Audio(out_wavs_after[0,0,:],rate=8000))
display(Audio(out_wavs_after[0,1,:],rate=8000))

In [ ]:
def step_test(start, results,model,loss_func,COMPUTE_METRICS,eval_save_dir,ex_save_dir,test_set,neptune_status,model_device,pretrained=False):
  series_list = []

  i = start

  torch.no_grad().__enter__()
  for idx in tqdm(range(len(test_set))):
      # Forward the network on the mixture.
      mix, sources, ids = test_set[idx]
      mix, sources = tensors_to_device([mix, sources], device=model_device)
      est_sources = model(mix.unsqueeze(0))
      loss, reordered_sources = loss_func(est_sources, sources[None], return_est=True)
      mix_np = mix.cpu().data.numpy()
      sources_np = sources.cpu().data.numpy()
      est_sources_np = reordered_sources.squeeze(0).cpu().data.numpy()

      utt_metrics = get_metrics(
                  mix_np,
                  sources_np,
                  est_sources_np,
                  sample_rate=conf["data"]["sample_rate"],
                  metrics_list=COMPUTE_METRICS,
              )


      r = 0
      iteration = str(i)
      neptune_logger.experiment.log_metric("Iteración:", i)

      for metric_name in COMPUTE_METRICS:
          input_metric_name = "input_" + metric_name     
          results[r] = results[r] + utt_metrics[input_metric_name]                   
          # iteration + "_"+
          neptune_logger.experiment.log_metric(input_metric_name, results[r])
          r = r +1
          

      for metric_name in COMPUTE_METRICS:
          results[r] = results[r] + utt_metrics[metric_name]          
          #iteration + "_"+ 
          neptune_logger.experiment.log_metric(metric_name, results[r])
          r = r +1
      
         
      #print("Result ")
      #print(results)
      #print("**"*23)
      utt_metrics["mix_path"] = test_set.mixture_path
      series_list.append(pd.Series(utt_metrics))


      i = i + 1 

      est_sources_np_normalized = normalize_estimates(est_sources_np, mix_np)


      # Save some examples in a folder. Wav files and metrics as text.
      if conf["test"]["n_save_examples"] == -1:
          conf["test"]["n_save_examples"] = len(test_set)
      
      save_idx = random.sample(range(len(test_set)),conf["test"]["n_save_examples"])


      if idx in save_idx:

          example_name = "ex_{}/".format(idx)

          local_save_dir = os.path.join(ex_save_dir, "ex_{}/".format(idx))
          os.makedirs(local_save_dir, exist_ok=True)
          
          
          sf.write(local_save_dir + "mixture.wav", mix_np, conf["data"]["sample_rate"])

        
          
          # Loop over the sources and estimates
          for src_idx, src in enumerate(sources_np):
              sf.write(local_save_dir + "s{}.wav".format(src_idx), src, conf["data"]["sample_rate"])
          for src_idx, est_src in enumerate(est_sources_np_normalized):
              path_estimation_source = local_save_dir + "s{}_estimate.wav".format(src_idx)

              if (pretrained):
                path_estimation_source = local_save_dir + "s{}_pretrained_estimate.wav".format(src_idx)
                              
              sf.write(path_estimation_source,
                      est_src,
                      conf["data"]["sample_rate"],
              )
              if (neptune_status):
                neptune_logger.experiment.log_artifact(path_estimation_source)


          #Send estimation wavs
          mix_path = local_save_dir + "mixture.wav"
          if (neptune_status):
            neptune_logger.experiment.log_artifact(mix_path)

          neptune_status = False
                  
          # Write local metrics to the example folder.
          with open(local_save_dir + "metrics.json", "w") as f:
              json.dump(utt_metrics, f, indent=0)
  
  return series_list

In [ ]:
def compute_global_metrics(start,end, series_list,COMPUTE_METRICS,eval_save_dir,pretrained=False):
  all_metrics_df = pd.DataFrame(series_list)
  name = "all_metrics.csv"
  final_metrics = "final_metrics.json"
  if (pretrained):
    name= "all_metrics_pretrained_model_start_"+str(start)+"_end_"+str(end)+".csv"
    final_metrics ="final_metrics_pretrained_model_start_"+str(start)+"_end_"+str(end)+".json"
    
  all_metrics_path = os.path.join(eval_save_dir,name )  
  all_metrics_df.to_csv(all_metrics_path)

  #Send All metrics
  neptune_logger.experiment.log_artifact(all_metrics_path)

  final_results = {}
  for metric_name in COMPUTE_METRICS:
      input_metric_name = "input_" + metric_name
      ldf = all_metrics_df[metric_name] - all_metrics_df[input_metric_name]
      final_results[metric_name] = all_metrics_df[metric_name].mean()
      final_results[metric_name + "_imp"] = ldf.mean()



  

  summary_metrics = os.path.join(eval_save_dir,final_metrics)


  with open(summary_metrics, "w") as f:
          json.dump(final_results, f, indent=0)

  #Send summary metrics
  neptune_logger.experiment.log_artifact(summary_metrics)

In [ ]:
def test(start, end, results, model,test_set,pretrained=False):
  model_device = next(model.parameters()).device
  loss_func = PITLossWrapper(pairwise_neg_sisdr, pit_from="pw_mtx")
  COMPUTE_METRICS = ["si_sdr", "sdr", "sir", "sar", "stoi"]

  eval_save_dir = os.path.join(exp_dir, "metrics")
  ex_save_dir = os.path.join(eval_save_dir, "examples/")
  neptune_status=True

  series_list = step_test(start, results, model,loss_func,COMPUTE_METRICS,eval_save_dir,ex_save_dir,test_set,neptune_status,model_device,pretrained)
  compute_global_metrics(start,end,series_list,COMPUTE_METRICS,eval_save_dir,pretrained)  
  neptune_logger.experiment.stop()

In [ ]:
neptune_logger.experiment.stop()

https://app.neptune.ai/josearangos/Tg-speech-separation/e/TGSPEEC-80
